In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import joblib

from pathlib import Path

In [2]:
ngt_static_landmarks = Path('./data/dataset/ngt_static_landmarks.csv')
ngt_static_classifier = Path('./data/models/ngt_static_classifier_normalized.pkl')



In [3]:
# Load data
df = pd.read_csv(ngt_static_landmarks)


def normalize_landmarks(row):
    """Normalize a row of landmark data"""
    # Extract coordinates as array (21 landmarks x 3 coords)
    coords = row.values.reshape(21, 3)
    
    # Center on wrist (landmark 0)
    wrist = coords[0]
    coords = coords - wrist
    
    # Scale by hand size
    hand_size = np.linalg.norm(coords[12] - coords[0])
    if hand_size > 0:
        coords = coords / hand_size
    
    return coords.flatten()

# Separate features and labels
X = df.drop('letter', axis=1)
y = df['letter']

# Normalize all samples
print("Normalizing training data...")
X_normalized = X.apply(normalize_landmarks, axis=1, result_type='expand')

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X_normalized, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")

# Train
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Evaluate
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"\nNormalized model trained!")
print(f"Accuracy: {accuracy:.2%}")
print(f"\nClassification Report:")
print(classification_report(y_test, y_pred))

# Save
joblib.dump(clf, ngt_static_classifier)
print(f"\nModel saved as {ngt_static_classifier}")

Normalizing training data...
Training samples: 605
Test samples: 152

Normalized model trained!
Accuracy: 98.68%

Classification Report:
              precision    recall  f1-score   support

           A       1.00      1.00      1.00         6
           B       1.00      1.00      1.00         6
           C       0.83      0.83      0.83         6
           D       1.00      1.00      1.00         6
           E       1.00      1.00      1.00         6
           F       1.00      1.00      1.00         6
           G       1.00      1.00      1.00        12
           I       1.00      1.00      1.00         6
           K       1.00      1.00      1.00        12
           L       1.00      1.00      1.00        12
           M       1.00      1.00      1.00        10
           N       1.00      1.00      1.00        10
           O       0.83      0.83      0.83         6
           P       1.00      1.00      1.00         6
           Q       1.00      1.00      1.00         

Starting real-time NGT recognition with normalized model...
Press 'q' to quit



I0000 00:00:1768848759.179814  619572 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 90.5), renderer: Apple M2
W0000 00:00:1768848759.192885  625233 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1768848759.198960  625235 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
